In [4]:
# Must contain this cell, in order to import from other folders
import os
import sys
sys.path.append(os.getcwd() + '/..')

%reload_ext autoreload
%autoreload 2

In [1]:
import pytorch_lightning as pl

# Set random seed
random_seed = 42
pl.seed_everything(random_seed)

# Create & fit the model using Pytorch Lightning's Trainer
trainer = pl.Trainer.from_argparse_args(args)
trainer.fit(model, datamodule=datamodule)

if args.fast_dev_run is None:
    trainer.test(datamodule=datamodule)

In [1]:
import torch.nn as nn

layer = nn.Linear(3, 2)

layer.in_features

3

In [25]:
from src.models.fe import FeatureExtractor

model = FeatureExtractor(backbone_name='vgg16', output_size=15)

model.backbone


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1